In [1]:
from PIL import Image
from pylab import *
import numpy as np
from scipy.ndimage import uniform_filter, gaussian_filter
import cv2

In [2]:
# 使用归一化的互相关计算视差图像
def ncc(im_l,im_r,start,steps,wid):
    m, n = im_l.shape
    # 保存不同求和值的数组
    mean_l = np.zeros((m, n))
    mean_r = np.zeros((m, n))
    s = np.zeros((m, n))
    s_l = np.zeros((m, n))
    s_r = np.zeros((m, n))
    # 保存深度平面的数组
    dmaps = np.zeros((m, n, steps))
    # 计算图像块的平均值
    uniform_filter(im_l, wid, mean_l)
    uniform_filter(im_r, wid, mean_r)
    # 归一化图像
    norm_l = im_l - mean_l
    norm_r = im_r - mean_r
    # 尝试不同的视差
    for displ in range(steps):
        #将左边图像移动到右边，计算加和
        uniform_filter(roll(norm_l, -displ - start) * norm_r, wid, s)  # 和归一化
        uniform_filter(roll(norm_l, -displ - start) * roll(norm_l, -displ - start), wid, s_l)
        uniform_filter(norm_r * norm_r, wid, s_r)  # 和反归一化
        # 保存ncc 的分数
        dmaps[:, :, displ] = s / np.sqrt((s_l * s_r) + 1e-5)
    # 为每个像素选取最佳深度
    return argmax(dmaps, axis=2)

In [3]:
def gauss(im_l,im_r,start,steps,wid):
    """ 使用带有高斯加权周边的归一化互相关计算视差图像"""
    m,n = im_l.shape
    # 保存不同加和的数组
    mean_l = zeros((m,n))
    mean_r = zeros((m,n))
    s = zeros((m,n))
    s_l = zeros((m,n))
    s_r = zeros((m,n))
    # 保存深度平面的数组
    dmaps = zeros((m,n,steps))
    # 计算平均值
    gaussian_filter(im_l,wid,0,mean_l)
    gaussian_filter(im_r,wid,0,mean_r)
    # 归一化图像
    norm_l = im_l - mean_l
    norm_r = im_r - mean_r
    # 尝试不同的视差
    for displ in range(steps):
        # 将左边图像移动到右边，计算加和
        gaussian_filter(roll(norm_l,-displ-start)*norm_r,wid,0,s) # 和归一化
        gaussian_filter(roll(norm_l,-displ-start)*roll(norm_l,-displ-start),wid,0,s_l)
        gaussian_filter(norm_r*norm_r,wid,0,s_r) # 和反归一化
        # 保存ncc 的分数
        dmaps[:,:,displ] = s/np.sqrt(s_l*s_r)
    # 为每个像素选取最佳深度
    return argmax(dmaps,axis=2)

In [4]:
im_l = array(Image.open('cones/im2.png').convert('L'), 'f')
im_r = array(Image.open('cones/im6.png').convert('L'), 'f')
# 开始偏移，并设置步长
steps = 40
start = 4
# ncc 的宽度
wids = [3, 9, 15]
for wid in wids:
    res1 = ncc(im_l, im_r, start, steps, wid)
    cv2.imwrite('cones/depth1-{}.png'.format(wid), res1)
    res2 = gauss(im_l, im_r, start, steps, wid)
    cv2.imwrite('cones/depth2-{}.png'.format(wid), res2)
